<a href="https://colab.research.google.com/github/lkmsdf159/Colleage/blob/main/multi_class_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# 데이터셋 경로 설정
train_dir = '/content/drive/MyDrive/data/train'
test_dir = '/content/drive/MyDrive/data/test'
val_dir = '/content/drive/MyDrive/data/validation'

# 이미지 사이즈 설정
target_size = (224, 224)

# 배치 사이즈 설정
batch_size = 16

# 데이터 전처리 및 증강
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 픽셀값을 0-1 범위로 정규화
    rotation_range=20,  # 회전 범위 (단위: 도)
    width_shift_range=0.1,  # 수평 이동 범위
    height_shift_range=0.1,  # 수직 이동 범위
    zoom_range=0.2,  # 확대/축소 범위
    horizontal_flip=True,  # 수평 반전
    vertical_flip=True,  # 수직 반전
    brightness_range=[0.8, 1.2],  # 밝기 범위
    fill_mode='nearest'  # 변형 후 빈 공간 채우기 방식
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# GPU 사용 설정
with tf.device('/device:GPU:0'):
    # 모델 정의
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

# 모델 저장
model.save('image_classifier.h5')

Found 131 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Found 18 images belonging to 3 classes.
Epoch 1/10
8/8 [==============================] - 29s 4s/step - loss: 1.0705 - accuracy: 0.8609 - val_loss: 0.6920 - val_accuracy: 0.7500
Epoch 2/10
8/8 [==============================] - 16s 2s/step - loss: 0.4270 - accuracy: 0.9217 - val_loss: 0.9854 - val_accuracy: 0.7500
Epoch 3/10
8/8 [==============================] - 16s 2s/step - loss: 0.4555 - accuracy: 0.9130 - val_loss: 0.9540 - val_accuracy: 0.8125
Epoch 4/10
8/8 [==============================] - 18s 2s/step - loss: 0.5190 - accuracy: 0.8984 - val_loss: 0.8919 - val_accuracy: 0.6875
Epoch 5/10
8/8 [==============================] - 15s 2s/step - loss: 0.3938 - accuracy: 0.9217 - val_loss: 1.2860 - val_accuracy: 0.6875
Epoch 6/10
8/8 [==============================] - 15s 2s/step - loss: 0.4413 - accuracy: 0.9043 - val_loss: 0.9781 - val_accuracy: 0.6875
Epoch 7/10
8/8 [==============================] - 1

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# 모델 평가
_, acc = model.evaluate(test_generator, steps=len(test_generator), verbose=0)
print('Test accuracy: %.3f' % (acc * 100.0))

loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Test accuracy: 46.667
1/1 [==============================] - 1s 878ms/step - loss: 1.9546 - accuracy: 0.4667
Test loss: 1.9546
Test accuracy: 0.4667
